In [1]:
import face_recognition
import cv2
import os
import pandas as pd
import numpy as np
import logging
import urllib3
import requests
from PIL import Image, ImageDraw 
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

%matplotlib inline

# OCTOPOD - Fetch Octo data

In [2]:
# SECRET KEYS! Contact octopod.project@octo.com for access to the Octopod API


token_url = 'https://octopod.octo.com/api/oauth/token'
post_data = {"grant_type": "client_credentials",
             "client_id": OCTOPOD_CLIENT_ID,
             "client_secret": OCTOPOD_CLIENT_SECRET}

client = BackendApplicationClient(client_id=OCTOPOD_CLIENT_ID)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url=token_url, client_id=OCTOPOD_CLIENT_ID, client_secret=OCTOPOD_CLIENT_SECRET)
base_url = 'https://octopod.octo.com/api/v0'

all_octo_response = requests.get(base_url + '/people', token)
all_octo = pd.read_json(all_octo_response.text)
all_octo.head(2)

,created_at,email,entry_date,first_name,id,included_in_activity_rate,job,last_name,leaving_date,lob,nickname,updated_at,url
0,2019-07-12 12:34:24,ronaldyne.philippe@octo.com,2019-07-22,Ronaldyne,2142665276,False,"{'id': 41, 'name': 'Admin', 'url': 'https://oc...",Philippe,None,"{'id': 122, 'abbreviation': 'EXT', 'url': 'htt...",RONP,2019-07-12 12:34:24,https://octopod.octo.com/api/v0/people/2142665...
1,2019-07-11 07:48:45,hpouilloux@octo.com,2019-07-15,Hugo,2142665275,False,"{'id': 41, 'name': 'Admin', 'url': 'https://oc...",Pouilloux,None,"{'id': 36, 'abbreviation': 'LAB', 'url': 'http...",HUGO,2019-07-12 08:22:15,https://octopod.octo.com/api/v0/people/2142665...


In [17]:
octo_ngrams = all_octo.nickname.apply(lambda x: x.lower())

In [21]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://askbob.octo.com/')

login_button = driver.find_element_by_id('google-signin-button')
login_button.click()

input_mail = driver.find_element_by_id('identifierId')
input_mail.send_keys('srochette@octo.com')

next_button = driver.find_element_by_id('identifierNext')
next_button.click()

password = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//input[@name='password']")))
password.send_keys('toto')

next_button = driver.find_element_by_id('passwordNext')
next_button.click()

#time.sleep(5)
import pdb;pdb.set_trace()

for ngram in octo_ngrams[:20]:
    driver.get(f'https://askbob.octo.com/users/{ngram}')
    driver.save_screenshot(f'./screenshots/{ngram}.png')

driver.close()

--Return--
> <ipython-input-21-6a7e676a7854>(20)<module>()->None
-> import pdb;pdb.set_trace()
(Pdb) c


In [26]:
y_1, y_2 = 270, 565
x_1, x_2 = 1892, 2185

for ngram in octo_ngrams[:20]:
    image = cv2.imread(f'./screenshots/{ngram}.png')
    face = image[y_1:y_2, x_1:x_2]
    cv2.imwrite(f'./screenshots/{ngram}.png', face)

In [ ]:
# Change the path to match the folder that contains your 'known faces'
PATH = './screenshots/'

known_face_names = []
known_face_encodings = []

for profile in os.listdir(PATH):
    if profile.endswith('.png'):
        name = profile.split('.')[0]
        
        # Create known face encodings array
        image = face_recognition.load_image_file(PATH + profile)
        try:    
            encoding = face_recognition.face_encodings(image)[0]
            known_face_encodings.append(encoding)     
            
            # Create known face names array
            known_face_names.append(name)
        except:
            continue

# OCTO Watch - OCTOPOD Face Recognition

In [6]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)

detected_octo = pd.DataFrame(columns=['NICKNAME', 'ID', 'TACE'])

while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            face_names.append(name)
            
            if not ((name in detected_octo['NICKNAME'].values) or name == "Unknown"):
                id_name = all_octo[all_octo['nickname']==name]['id'].iloc[0]
                
                # You may change the end of the URL (dates are currently hard coded)
                tace_response = requests.get(
                    base_url+'/people/'+str(id_name)+'/activity_rate?from_date=01%2F01%2F2019&to_date=07%2F03%2F2019&include_pipe=false', token)
                tace_name = tace_response.json()['value']
                detected_octo = detected_octo.append({'NICKNAME': name,'ID': id_name, 'TACE': tace_name},ignore_index=True)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (210, 188, 93), 2)
        
        cv2.rectangle(frame, (left, top + 30), (right, top), (210, 188, 93), cv2.FILLED)
        cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (210, 188, 93), cv2.FILLED)

        font = cv2.FONT_HERSHEY_DUPLEX
        
        if name != "Unknown":
            
            tace_raw = detected_octo[detected_octo['NICKNAME'] == name]['TACE'].values[0] * 100
            tace_text = 'TACE: ' + '{0:.2f}%'.format(tace_raw)
            
            cv2.putText(frame, name, (left + 6, top + 24), font, 0.7, (255, 255, 255), 1)
            cv2.putText(frame, tace_text, (right - 160, bottom - 8), font, 0.7, (255, 255, 255), 1)

        else:
            cv2.putText(frame, name, (left + 6, top + 24), font, 0.7, (255, 255, 255), 1)

    cv2.imshow('OCTO Watch', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()

### The end